In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import cv2 as cv
import argparse
from PIL import Image, ImageEnhance, ImageDraw 
import matplotlib.pyplot as plt
import os

In [12]:
def ROI(frame):
    #enhancer = ImageEnhance.Contrast(frame)
    #img = enhancer.enhance(0.7)
    face_classifier = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.1, 5)
    x=0
    y=0
    w=0
    h=0
    max_area = 0
    for face in faces:
        x_t, y_t, w_t, h_t = face
        if w_t*h_t>max_area:
            max_area = w_t*h_t
            x,y,w,h = face
    #print(faces)
    
    return x,y,w,h

In [4]:
import keras
model = keras.models.load_model("model_trained")

In [5]:
emotion = {
    '0' : 'Afraid',
    '1' : 'Angry',
    '2' : 'Disgust',
    '3' : 'Happy',
    '4' : 'Nervous',
    '5' : 'Sad',
    '6' : 'Surprise'
}

In [13]:
def using_cam():
    cap = cv.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        #cv.imshow("Input",frame)
        #frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        x,y,w,h = ROI(frame)
        if x==0 and y==0 and w==0:
            continue
        #cv.rectangle(frame, (x,y), (x+w, y+h), (127, 0, 255), 2)
        croped_image = frame[x:x+w, y:y+h]
        croped_image = cv.cvtColor(croped_image, cv.COLOR_BGR2RGB)
        cv.imshow('ROI', croped_image)
        if(cv.waitKey(2) == 13 & 0xFF):
            break
        
        
        img = Image.fromarray(frame)
        img.save("Pic.jpg")
        img = keras.preprocessing.image.load_img(
            'Pic.jpg', target_size=(128, 128)
        )
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0) # Create a batch
        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])
        
        font = cv.FONT_HERSHEY_SIMPLEX
        fontScale = 2
        thickness = 4
        color = (255, 0, 0)
        pos = (x, y+h)
        image = cv.putText(frame, emotion[str(np.argmax(score))], pos, font, fontScale, color, thickness, cv.LINE_AA)
        
        
        #print(emotion[str(np.argmax(score))])
        cv.imshow("Final", image)
    cap.release()
    cv.destroyAllWindows()

In [14]:
using_cam()